# 电机异音AI诊断

本项目为飞桨领航团AI达人创造营第二期作业。

## 项目背景
在电机生产线上普遍采用人工听音的方法分辨良、次品，不仅成本高，而且重复、单调的听音工作极易引起人员疲劳，容易出现误判，若个别不良品混入整批成品中，会给工厂带来严重经济损失，甚至严重影响产品声誉。

（数据对应的比赛）本次大赛要求参赛者基于加速度传感器采集的振动信号，利用机器学习、深度学习等人工智能技术，设计智能检验的算法，要求算法对故障电机不能有漏识别，在召回100%的情况下，尽量提高预测准确率，以达到替代人工质检的目的。

## 数据说明

文件清单：

1. Motor_tain.zip：用于训练的采集数据，其中，文件夹“正样本”包含30个异常电机的数据样本，文件夹“负样本”包含500个正常电机的数据样本；
2. Motor_testP.zip：用于测试的采集数据，包含500个电机的数据样本；  
文件说明：采集数据时是分别对电机正转、反转时的振动信号进行采集。也就是说每台电机有两条数据，其中F代表正转，B代表反转。每条数据包含两路振动信号，数据文件命名规则：编号_旋转方向.csv。比赛链接：http://jingsai.julyedu.com/v/25820185621432447/dataset.jhtml

## 项目方案

每个样本包含F和B两份csv，每个csv文件包含79999×2个数据。使用多个全连接层进行预测。使用的损失函数为交叉熵。

模型如下：

![](https://ai-studio-static-online.cdn.bcebos.com/d1303434dd3a4d0ba2c23c5146be9fe0648c0389ac2741729aaefd97a3855118)

## 处理方案

1. 重采样：针对数据不平衡，复制多份正向样本，从而训练时达到类别均衡采样的效果
2. 修改损失：针对题目要求的保证TP最大，尽可能减小FP的问题，将label==1的损失权重设定为1.1。

# 代码

## 预处理

In [1]:
! unzip -oq data/data31822/Motor_tain.zip
! unzip -oq data/data31822/Motor_testP.zip

In [31]:
import os

train_pos_dir='Motor_tain/╒¤╤∙▒╛/'
train_neg_dir='Motor_tain/╕║╤∙▒╛/'

# 生成三份文件便于按照类别比例划分
with open('train_pos.txt','w') as f:
    for item in list(set([item[:len(item)-6] for item in os.listdir(train_pos_dir)])):
        if '.ipynb' not in item:
            f.write(train_pos_dir+item+'\t1\n')
with open('train_neg.txt','w') as f:
    for item in list(set([item[:len(item)-6] for item in os.listdir(train_neg_dir)])):
        if '.ipynb' not in item:
            f.write(train_neg_dir+item+'\t0\n')

with open('train.txt','w') as f:
    for item in list(set([item[:len(item)-6] for item in os.listdir(train_pos_dir)])):
        if '.ipynb' not in item:
            for i in range(int(500/30)):
                f.write(train_pos_dir+item+'\t1\n')
    for item in list(set([item[:len(item)-6] for item in os.listdir(train_neg_dir)])):
        if '.ipynb' not in item:
            f.write(train_neg_dir+item+'\t0\n')

## 构造读取器

In [33]:
import paddle
import numpy as np
import paddle.vision.transforms as T
from PIL import Image
import pandas as pd

class MyDateset(paddle.io.Dataset):
    def __init__(self,txt_dir):
        super(MyDateset, self).__init__()
        
        self.path=[]
        self.label=[]

        with open(txt_dir,'r') as f:
            for line in f.readlines():
                self.path.append(line.split('\t')[0])
                self.label.append(line.split('\t')[1][0])

    def __getitem__(self, index):
        path = self.path[index]
        label = int(self.label[index])

        F=pd.read_csv(path+'_F.csv')
        B=pd.read_csv(path+'_B.csv')

        F=paddle.to_tensor(F.values).flatten(0).astype('float32')
        B=paddle.to_tensor(B.values).flatten(0).astype('float32')

        label = np.array(label).astype('int64')

        return F,B,label

    def __len__(self):
        return len(self.label)

In [34]:
train_dataset=MyDateset('train.txt')

train_dataloader = paddle.io.DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=True,
    drop_last=False)

for step, data in enumerate(train_dataloader):
    F, B, label = data
    print(step, F.shape, B.shape, label.shape)
    break

0 [16, 159998] [16, 159998] [16]


## 构造网络模型

In [35]:
class MyNet(paddle.nn.Layer):
    def __init__(self):
        super(MyNet,self).__init__()
        self.fc_F = paddle.nn.Linear(in_features=159998, out_features=2000)
        self.fc_B = paddle.nn.Linear(in_features=159998, out_features=2000)

        self.fc_1  = paddle.nn.Linear(in_features=4000, out_features=1000)
        self.fc_2  = paddle.nn.Linear(in_features=1000, out_features=200)
        self.fc_3  = paddle.nn.Linear(in_features=200, out_features=2)

    def forward(self,F,B):
        F = self.fc_F(F)
        B = self.fc_B(B)
        x = paddle.concat([F,B],axis=-1)
        x = self.fc_1(x)
        # x = paddle.nn.functional.relu(x)
        x = self.fc_2(x)
        # x = paddle.nn.functional.relu(x)
        x = self.fc_3(x)
        return x

## 训练

In [38]:
model = MyNet()
model.train()

max_epoch=10
opt = paddle.optimizer.SGD(learning_rate=0.001, parameters=model.parameters())

now_step=0
for epoch in range(max_epoch):
    for step, data in enumerate(train_dataloader):
        now_step+=1

        F, B, label = data
        pre = model(F,B)
        loss = paddle.nn.functional.cross_entropy(pre,label,weight=paddle.to_tensor([1,1.1]),reduction='mean')
        loss.backward()
        opt.step()
        opt.clear_gradients()
        if now_step%100==0:
            print("epoch: {}, batch: {}, loss is: {}".format(epoch, step, loss.mean().numpy()))

paddle.save(model.state_dict(), 'model.pdparams')

epoch: 1, batch: 37, loss is: [0.631444]
epoch: 3, batch: 13, loss is: [0.63104683]
epoch: 4, batch: 51, loss is: [0.63319224]
epoch: 6, batch: 27, loss is: [0.5660453]
epoch: 8, batch: 3, loss is: [0.5968682]
epoch: 9, batch: 41, loss is: [0.5927209]


## 查看混淆矩阵

格式为


|  | 预测为0 | 预测为1 |
| -------- | -------- | -------- |
| 实际为0     | value    | value     |
| 实际为1     | value     | value     |



In [39]:
# mydict = paddle.load("model_1.pdparams")
# model.set_state_dict(mydict)

record=np.zeros([2,2])
for i in range(len(train_dataset)):
    F,B,label=train_dataset[i]
    pre=model(F,B)
    # print(f'real label: {label} pre label: {np.argmax(pre.numpy())}')
    record[label.tolist()][np.argmax(pre.numpy())]+=1
record

array([[455.,  45.],
       [  0., 480.]])

# 总结

本项目使用了极简方式构造了一个电机异音AI诊断模型，最终结果表明模型可以将全部正样本（异常）都判断正确，并且负样本的占比为9%。除去模型效果的因素外，本项目有以下不足，可以继续改进：

1. 本项目构造了一个简单的全连接层网络模型，但由于数据量较大，需要在第一个全连接层尽可能地压缩节点数。可以尝试使用卷积替换全连接层。
3. 由于样本量较少（正向样本仅30个），仅设置了训练集，没有设定验证集。

# 个人介绍

> 作者：笠雨聆月

> 兴趣：目前从最容易上手的cv进行学习，也在尝试nlp，gan等等，各种方向来者不拒

> 个人主页：https://aistudio.baidu.com/aistudio/personalcenter/thirdview/608082

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 